In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import os
import json
tfk  = tf .keras
tfkl = tfk.layers

from utils import *
from guided_ig import *
from tqdm import tqdm
from sklearn.metrics import auc
import pickle

from tensorflow.python.ops import image_ops
from tensorflow.python.ops import io_ops
from tensorflow.python.keras.layers.preprocessing import image_preprocessing

In [2]:
p_xml = '/media/big/imagenet/data/ILSVRC/Annotations/CLS-LOC/val'
p_jpg = '/media/big/imagenet/data/ILSVRC/Data/CLS-LOC/val'
files_xml = sorted(os.listdir(p_xml))[:5000]
files_jpg = sorted(os.listdir(p_jpg))[:5000]

In [3]:
with open('imagenet_class_index.json', 'r') as fp:
    label_json = json.loads(fp.read())
labels = np.array([
    [k, *v]
    for k, v in label_json.items()
])

In [4]:
model = tf.keras.applications.InceptionV3()
def grad_func(x, idx):
    x = tf.constant(x)[tf.newaxis]
    with tf.GradientTape() as tape:
        tape.watch(x)
        pred = model(x)
        pred = pred[:, idx]
    return np.squeeze(tape.gradient(pred, x).numpy())

2021-12-08 23:11:39.036497: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-08 23:11:39.044231: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-08 23:11:39.044615: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-08 23:11:39.045546: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
p_save = '/media/big/imagenet/computed-gig'

for f_xml, f_jpg in tqdm(zip(files_xml, files_jpg,
                            # Comment to run
                            #[]
                            )): 
    with open(f'{p_xml}/{f_xml}') as fp:
        xml = fp.read()

    path = f'{p_jpg}/{f_jpg}'
    img = io_ops.read_file(path)
    img = image_ops.decode_image(
        img, 
        channels = 3, 
        expand_animations = False
    )
    img = image_ops.resize_images_v2(
        img, 
        (299, 299), 
        method = image_preprocessing.get_interpolation('bilinear')
    )
    img = preprocess_iv3(img)[0]
    p5  = model(img[tf.newaxis]).numpy().flatten().argsort()[::-1][:5]
    y   = xml.split('\n')[13][8:-7]
    
    check_match = np.argwhere(labels[p5, 1] == y)
    if len(check_match) > 0:
        check_idx = p5[check_match][0, 0]
        
        # Guided IG
        gig = np.stack([
            unbounded_guided_ig(
                img.numpy(),
                baseline, 
                200, 
                lambda x: grad_func(x, check_idx), 
                0.1
            )
            for baseline in [
                np.zeros_like(img) - 1, # To the model, black is -1
                np.ones_like (img)      # White is 1
            ]
        ])
        
        # Anchored Guided IG (GIG(20))
        gig20 = np.stack([
            anchored_guided_ig(
                img.numpy(),
                baseline, 
                lambda x: grad_func(x, check_idx), 
                200, 
                0.1,
                anchors = 20
            )
            for baseline in [
                np.zeros_like(img) - 1,
                np.ones_like (img)
            ]
        ])
        res = np.concatenate([gig, gig20])
        
        with open(f'{p_save}/{f_xml}-{f_jpg}.pickle', 'wb') as fp:
            pickle.dump({
                'res': res,
                'p5' : p5,
                'y'  : y
            }, fp)

0it [00:00, ?it/s]2021-12-08 23:11:53.675393: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2021-12-08 23:11:53.842423: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
0it [00:00, ?it/s]


In [6]:
def bbox_plot(ax, xmin, xmax, ymin, ymax):
    ax.plot([xmin, xmax], [ymin, ymin], 'r')
    ax.plot([xmin, xmin], [ymin, ymax], 'r')
    ax.plot([xmin, xmax], [ymax, ymax], 'r')
    ax.plot([xmax, xmax], [ymin, ymax], 'r')

In [33]:
inbox = lambda samples, xmin, xmax, ymin, ymax: (
    (samples > [xmin, ymin]) & 
    (samples < [xmax, ymax])
).all(1)

idxs = np.array([
    (idx, jdx)
    for idx in range(299)
    for jdx in range(299)
])

saved_res = os.listdir(p_save)
ress = []
for sr in tqdm(saved_res):
    
    xml, jpg = sr[:-7].split('-')
    
    xml = f'{p_xml}/{xml}'
    jpg = f'{p_jpg}/{jpg}'
    res = f'{p_save}/{sr}'
    
    with open(res, 'rb') as fp:
        obj = pickle.load(fp)
    
    # Make sure that bounding box is actually available
    if labels[obj['p5'][0], 1] == obj['y']:
    
        # Load image
        img = io_ops.read_file(jpg)
        img = image_ops.decode_image(
            img, 
            channels = 3, 
            expand_animations = False
        )
        img = image_ops.resize_images_v2(
            img, 
            (299, 299), 
            method = image_preprocessing.get_interpolation('bilinear')
        )
        img.set_shape((299, 299, 3))
        img = tfk.applications.inception_v3.preprocess_input(img)

        # Load BBOX
        as_string = tf.io.read_file(xml)
        vals = tf.gather(tf.strings.split(as_string, '\n'), [7, 8, 18, 19, 20, 21])
        vals = tf.strings.regex_replace(vals, '[^0123456789]', '')
        vals = tf.strings.to_number(vals, tf.int32)

        xmin, xmax, ymin, ymax = (
            np.round(299 * vals[2] / vals[0]),
            np.round(299 * vals[4] / vals[0]),
            np.round(299 * vals[3] / vals[1]),
            np.round(299 * vals[5] / vals[1]),
        )
        
        # Compute IG
        
        best_idx   = obj['p5'][0]
        image      = img[tf.newaxis]
        baseline_b = tf.zeros_like(image) - 1
        baseline_w = tf .ones_like(image)

        steps = 200
        batch = 25
        igs   = []
        for baseline in [baseline_b, baseline_w]:
            delta  = (image - baseline) / steps
            alpha  = tf.cast(tf.linspace(0, steps, steps)[:, tf.newaxis, tf.newaxis, tf.newaxis], tf.float32)
            inputs = baseline + (alpha * delta)

            ig = np.zeros_like(image)
            pointer = 0
            while pointer < steps:
                inp = tf.gather(inputs, tf.range(pointer, pointer + batch))
                with tf.GradientTape() as tape:
                    tape.watch(inp)
                    pred = model(inp)
                    pred = pred[:, best_idx]
                grad = tape.gradient(pred, inp)
                ig += tf.reduce_sum(grad, 0).numpy()
                pointer += batch
            igs.append(ig)

        # Prepare attributions
        all_attr = (
            obj['res'][0].mean(-1),           # GIG B
            obj['res'][1].mean(-1),           # GIG W
            obj['res'][[0, 1]].mean((0, -1)), # GIG B + W
            obj['res'][2].mean(-1),           # GIG 20 B
            obj['res'][3].mean(-1),           # GIG 20 W
            obj['res'][[2, 3]].mean((0, -1)), # GIG 20 B + W
            igs[0].mean((0, -1)),             # IG B
            igs[1].mean((0, -1)),             # IG W
            np.concatenate(igs).mean((0, -1)),# IG B + W 
            grad[-1].numpy().mean(-1),        # Gradient
            (grad[-1] * img).numpy().mean(-1) # Gradient x Image
        )

        # Compute AUROC
        qs = np.linspace(1, 0, 5)
        res = []
        for attr in all_attr:
            auc_ = []
            for q in qs:

                positive_attr = np.concatenate([
                    idxs, 
                    tf.gather_nd(attr, idxs).numpy()[:, np.newaxis]
                ], 1)
                positive_attr = positive_attr[positive_attr[:, 2] > 0]
                
                pos = np.argwhere(positive_attr[:, 2] >= np.quantile(positive_attr[:, 2], q)).flatten()
                neg = np.argwhere(positive_attr[:, 2] <  np.quantile(positive_attr[:, 2], q)).flatten()
                TP = inbox(positive_attr[pos, :2], xmin, xmax, ymin, ymax)
                FP = ~TP
                FN = inbox(positive_attr[neg, :2], xmin, xmax, ymin, ymax)
                TN = ~FN

                TP = TP.sum()
                TN = TN.sum()
                FP = FP.sum()
                FN = FN.sum()
                
                auc_.append((FP / (FP + TN), TP / (TP + FN)))
                
            auc_ = np.array(auc_).T
            res.append(auc(auc_[0], auc_[1]))
            
        ress.append(res)
    
        
ress = pd.DataFrame(ress, columns = [
    'GIG(0) (Black)',
    'GIG(0) (White)',
    'GIG(0) (B + W)',
    'GIG(20) (White)',
    'GIG(20) (Black)',
    'GIG(20) (B + W)',
    'IG (Black)',
    'IG (White)',
    'IG (Black + White)',
    'Gradient',
    'Gradient x Image'
])

 14%|█████▎                                | 431/3100 [27:45<2:41:02,  3.62s/it]/tmp/ipykernel_4347/1580975637.py:124: RuntimeWarning: invalid value encountered in long_scalars
  auc_.append((FP / (FP + TN), TP / (TP + FN)))
100%|█████████████████████████████████████| 3100/3100 [3:20:21<00:00,  3.88s/it]


In [34]:
ress.describe()

,GIG(0) (Black),GIG(0) (White),GIG(0) (B + W),GIG(20) (White),GIG(20) (Black),GIG(20) (B + W),IG (Black),IG (White),IG (Black + White),Gradient,Gradient x Image
count,2548.000000,2550.000000,2550.000000,2548.000000,2550.000000,2550.000000,2550.000000,2550.000000,2550.000000,2550.000000,2550.000000
mean,0.633722,0.647112,0.650166,0.631799,0.634139,0.644910,0.630253,0.629399,0.631751,0.605682,0.586419
std,0.110025,0.105562,0.107495,0.099853,0.103880,0.102716,0.097576,0.095306,0.096705,0.093833,0.095565
min,0.185754,0.163102,0.168265,0.236575,0.181559,0.231746,0.237349,0.249118,0.236913,0.242713,0.217377
25%,0.560989,0.581348,0.580452,0.573533,0.571482,0.584462,0.574038,0.572563,0.573752,0.546212,0.526636
50%,0.639647,0.650414,0.656344,0.639480,0.643722,0.656327,0.633224,0.633024,0.635984,0.605109,0.586540
75%,0.710265,0.719867,0.724181,0.701026,0.706197,0.714177,0.692376,0.690881,0.693606,0.665088,0.650118
max,0.883116,0.880170,0.883599,0.881073,0.876854,0.860696,0.875525,0.871978,0.874498,0.866244,0.864503


In [36]:
ress.to_csv('results.csv')